In [ ]:
!pip install langchain-teddynote markdownify pymupdf

In [9]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("Jimin-Parser-Test")

LangSmith 추적을 시작합니다.
[프로젝트명]
Jimin-Parser-Test


In [3]:
import os
import pymupdf
from glob import glob
import json
import requests
from PIL import Image

In [6]:
SAMPlE = "data/sample-report.pdf"

아무리 좋은 parser라도 pdf 문서 안의 이미지, 표 제대로 처리 안됨.

- 업스테이지 : layout parsing algorithm (가격) 무료 크레딧 30달러
- 무료 : 찾아보는 ing

## layout parsing algorithm

레이아웃에서 가져온 이미지를 벡터디비에 넣고, 이미지를 다시 우리한테 꺼내서 보여줘야할 줄 알아야함. layout parser는 이 영역의 이미지가 존재한다를 알려주는거지, 이 이미지를 따로 따서 저장하지는 않음. 

- table도 이미지 캡처떠서 멀티 모달에 넣어서 그거에 마크다운 변환해서 넣어주면 성능이 좋아짐. 



In [10]:
filepath = SAMPlE
input_pdf = pymupdf.open(filepath)
num_pages = len(input_pdf)
print(f"총 페이지 수: {num_pages}")

총 페이지 수: 22
